In [12]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [51]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
dataorig = pd.read_excel("C:/Users/Usuario/Documents/Projectes/ChallengeAuditoria/challenge-auditoria/data/inventory_data_new.xlsx")

dataorig["cost_valor"] = dataorig["preu_venda_unitari_2023"] / dataorig["cost_unitari_stock_2023"]
# Drop rows with missing values

data = dataorig.copy()
data.dropna(inplace=True)


# Encode categorical variables if needed
# (If there are any categorical variables that need encoding)

# Select relevant features for the Autoencoder
features = [ 'proporcio_variacio_preu_venda_unitari_2022_2023', 'diferencia_entrada_sortida', 'dies_ultima_sortida', 'stock_final_2023', 'cost_valor']

X = data[features]

# Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
X_train, X_val = train_test_split(X_scaled, test_size=0.2, random_state=42)

print("Shape of training data:", X_train.shape)
print("Shape of validation data:", X_val.shape)


Shape of training data: (233, 5)
Shape of validation data: (59, 5)


In [38]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

# Define Autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 4  # Increase the size of the encoding layer

input_layer = Input(shape=(input_dim,))
encoder = Dense(32, activation='relu')(input_layer)  # Add more neurons
encoder = Dense(encoding_dim, activation='relu')(encoder)
decoder = Dense(32, activation='relu')(encoder)  # Add more neurons
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder model
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(X_val, X_val))

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 1.0524 - val_loss: 0.7411
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.9676 - val_loss: 0.7156
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.2190 - val_loss: 0.6940
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.8814 - val_loss: 0.6754
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0822 - val_loss: 0.6585
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.7602 - val_loss: 0.6452
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7052 - val_loss: 0.6322
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.0117 - val_loss: 0.6181
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8866 - val_loss: 0.6042
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.8508 - val_loss: 0.5914
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7669 - val_loss: 0.5807
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7505 - val_loss: 0.5716
Epoc

In [39]:
import tensorflow as tf

# Assuming reconstructions and X_val are numpy arrays
reconstructions = autoencoder.predict(X_val)
mse = tf.keras.losses.mean_squared_error(X_val, reconstructions)

# Calculate reconstruction errors (no need to specify axis for 1D array)
mse_numpy = mse.numpy()
reconstruction_errors = mse_numpy


# Set a threshold for anomaly detection
threshold = reconstruction_errors.mean() + 2 * reconstruction_errors.std()

# Identify assets with reconstruction errors above the threshold
anomalies_indices = [i for i, error in enumerate(reconstruction_errors) if error > threshold]
anomalies = data.iloc[anomalies_indices]

print("Assets needing depreciation:")
print(anomalies)
print(len(anomalies))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Assets needing depreciation:
       material  unitats_2022  vendes_2022  preu_venda_unitari_2022  \
90  133066450.0      504000.0     17266.32                 0.034259   

    unitats_2023  vendes_2023  preu_venda_unitari_2023  \
90     1135800.0     35866.95                 0.031579   

    variacio_preu_venda_unitari_2022_2023  \
90                               -0.00268   

    proporcio_variacio_preu_venda_unitari_2022_2023 data_darrera_entrada  \
90                                        -0.078228           2023-09-27   

    dies_ultima_entrada data_darrera_sortida  dies_ultima_sortida  \
90                 95.0           2023-10-19                 73.0   

    diferencia_entrada_sortida  stock_final_2023  valor_total_stock_2023  \
90                        22.0            3600.0                   47.66   

    cost_unitari_stock_2023  cost_valor  
90                 0.013239    2.385289  
1


In [52]:
# Define accounting standards
salvage_value = 0  # Assumption: Salvage value is 0 (can be adjusted)
original_cost = dataorig['cost_unitari_stock_2023']  # Original cost of the asset

# Calculate the depreciation expense using the straight-line method
# We need to estimate the useful life or make an assumption
# For demonstration, let's assume a useful life of 5 years
useful_life_years = 5
depreciation_expense = (original_cost - salvage_value) / useful_life_years

# Define an acceptable range for depreciation value
# We can use a percentage of the original cost or a fixed value
# For demonstration, let's use a percentage (e.g., 10%) of the original cost
acceptable_range_percentage = 0  # 10%
acceptable_depreciation_range = original_cost * acceptable_range_percentage

# Calculate the lower and upper bounds of the acceptable range
lower_bound = original_cost - acceptable_depreciation_range
upper_bound = original_cost + acceptable_depreciation_range

# Determine assets that fall outside the acceptable range
outside_acceptable_range = (dataorig['preu_venda_unitari_2023'] < lower_bound)
assets_outside_range = dataorig[outside_acceptable_range]

print("Assets needing depreciation beyond acceptable range:")
print(assets_outside_range)


Assets needing depreciation beyond acceptable range:
        material  unitats_2022  vendes_2022  preu_venda_unitari_2022  \
10   115030252.0       75900.0      6825.62                 0.089929   
100  133066532.0           NaN          NaN                      NaN   
124  133100502.0      240000.0     13191.12                 0.054963   
453  152270347.0           NaN          NaN                      NaN   
569  153010114.0      723800.0     24060.01                 0.033241   
570  153010115.0     7937600.0    320326.09                 0.040356   
607  153110150.0    35825400.0    812679.69                 0.022684   
608  153110151.0     2906800.0     63774.72                 0.021940   
612  153110155.0     2756000.0     58779.76                 0.021328   
614  153110157.0     7384000.0    168335.44                 0.022797   
962  173430205.0    11835000.0    413846.10                 0.034968   
982  182040131.0      332800.0      8366.59                 0.025140   

     unita